<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import pandas as pd
import numpy as np
import os

The `os` library is part of the Python standard library. It ships with Python. It is extremely useful for performing system commands from within Python. 

1. Create an `output` folder using `os.mkdir()`
**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

2. Use `os.listdir()` to create a variable called `files` that is the list of files in the `data` folder.



In [2]:
# Create an output folder.
os.mkdir('output')

# Create a files variable that contains all of our data files.
files = os.listdir('./data')

### Step 2: Process one DataFrame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [3]:
# Read in data from your city from January 1st.
jan_1 = pd.read_excel('./data/Jan 1.xlsx', sheet_name='San Francisco')
jan_1

/Users/andresperez/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,prodcode,price_eu,weight_kg,quantity
0,4081,1.396291,1.897559,287.0
1,4062,1.845555,4.361009,190.0
2,4030,2.046969,1.709318,181.0
3,4650,1.117934,3.035264,121.0
4,4796,1.641424,9.884732,334.0
...,...,...,...,...
189,4081,2.263487,4.647049,137.0
190,4159,1.860518,8.239610,147.0
191,4272,1.347941,8.154360,280.0
192,4134,2.340641,4.469787,381.0


### Step 2a: Convert to units common in the USA
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and use 1.1 dollars per euro.

In [4]:
jan_1['price_usd'] = jan_1['price_eu']*1.1
jan_1['weight_lb'] = jan_1['weight_kg']*2.2

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [5]:
plu = pd.read_csv("plu-codes.csv")
plu.rename(columns={'plu_code': 'prodcode'}, inplace=True)
plu.head(2)

,product,prodcode
0,Apple (Fuji),4131
1,Apple (Gala),4134


In [6]:
jan_1 = pd.merge(jan_1, plu)

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [7]:
jan_1.drop(columns=['price_eu', 'weight_kg'], inplace=True)

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [8]:
jan_1['date'] = 'Jan 1'

In [9]:
jan_1

,prodcode,quantity,price_usd,weight_lb,product,date
0,4081,287.0,1.535920,4.174629,Eggplant,Jan 1
1,4081,487.0,0.830877,4.543806,Eggplant,Jan 1
2,4081,260.0,2.209687,17.764996,Eggplant,Jan 1
3,4081,269.0,2.437335,19.857462,Eggplant,Jan 1
4,4081,244.0,2.750768,17.810385,Eggplant,Jan 1
...,...,...,...,...,...,...
189,4404,274.0,2.503808,13.234113,Peach,Jan 1
190,4404,451.0,2.136732,21.041611,Peach,Jan 1
191,4404,276.0,1.077724,7.581778,Peach,Jan 1
192,4404,123.0,1.444505,2.945820,Peach,Jan 1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [10]:
def process_data(file, city):
    path = './data/' + str(file)
    df = pd.read_excel(path, sheet_name=city)
    
    df['price_usd'] = df['price_eu']*1.1
    df['weight_lb'] = df['weight_kg']*2.2

    plu = pd.read_csv("plu-codes.csv")
    plu.rename(columns={'plu_code': 'prodcode'}, inplace=True)
    df = pd.merge(df, plu)

    df.drop(columns=['price_eu', 'weight_kg'], inplace=True)

    df['date'] = str(file).strip('.xlsx')

    df['city'] = city

    df = df.reindex(columns=['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd'])

    return df

Test your function out on a new file and city!

In [11]:
process_data('Jan 2.xlsx', 'Boston')

/Users/andresperez/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,city,date,product,prodcode,quantity,weight_lb,price_usd
0,Boston,Jan 2,Kiwi,4030,109.0,16.298007,1.974055
1,Boston,Jan 2,Kiwi,4030,276.0,17.342691,2.279744
2,Boston,Jan 2,Kiwi,4030,295.0,12.269643,1.812637
3,Boston,Jan 2,Kiwi,4030,295.0,2.346695,0.850724
4,Boston,Jan 2,Kiwi,4030,127.0,6.389678,0.734666
...,...,...,...,...,...,...,...
141,Boston,Jan 2,Onion,4159,190.0,6.011113,2.554186
142,Boston,Jan 2,Onion,4159,309.0,8.774159,0.714954
143,Boston,Jan 2,Onion,4159,345.0,4.888816,1.252992
144,Boston,Jan 2,Onion,4159,431.0,8.623570,2.551195


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [12]:
df_list = [process_data(file, 'San Francisco') for file in files]

/Users/andresperez/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [13]:
pd.concat(df_list)

,city,date,product,prodcode,quantity,weight_lb,price_usd
0,San Francisco,Jan 23,Lemon,4958,197.0,11.363450,1.946762
1,San Francisco,Jan 23,Lemon,4958,284.0,20.465283,1.150748
2,San Francisco,Jan 23,Lemon,4958,186.0,14.406286,1.266580
3,San Francisco,Jan 23,Lemon,4958,460.0,6.620200,1.323316
4,San Francisco,Jan 23,Lemon,4958,325.0,12.182408,0.822722
...,...,...,...,...,...,...,...
145,San Francisco,Jan 8,Tomato,4796,264.0,14.220020,1.549764
146,San Francisco,Jan 8,Tomato,4796,406.0,2.923254,1.080412
147,San Francisco,Jan 8,Tomato,4796,244.0,7.547187,2.698783
148,San Francisco,Jan 8,Tomato,4796,395.0,16.861293,2.642471


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|


In [14]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [15]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

for k, v in city_dict.items():
    df_list = [process_data(file, k) for file in files]
    file = pd.concat(df_list)
    filepath = './output/' + v
    file.to_csv(filepath)


/Users/andresperez/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list.
1. Combine all of those DataFrames into one large DataFrame.
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [16]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!
files = files = os.listdir('./output')

for k, v in city_dict.items():
    df_list = [pd.read_csv('./output/' + file) for file in files]

data = pd.concat(df_list)

data.groupby('city').mean()


,Unnamed: 0,prodcode,quantity,weight_lb,price_usd
city,,,,,
Atlanta,79.285264,4221.003568,302.485306,11.929027,1.937980
Austin,78.272553,4226.061277,301.048298,12.092775,1.937456
Boston,75.818301,4232.272331,298.806536,12.063057,1.900973
Chicago,77.816602,4227.265551,301.686186,12.101555,1.930026
Denver,82.305483,4232.204734,301.012745,12.129850,1.932088
Los Angeles,79.760978,4226.015120,301.531276,12.167527,1.918331
New York,76.993964,4224.536969,299.482863,12.090294,1.912662
San Francisco,80.051518,4224.431414,298.979895,12.237399,1.925830
Seattle,76.541062,4238.488624,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Export Notebook as as > Executable Script` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).